In [ ]:
import findspark
findspark.init('/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType

# Sesion

Lo primero siempre es crear la sesion de Spark. La sesion permite a todos los procesos involucrados compartir contexto.

In [ ]:
spark = SparkSession.builder.appName("movielens").getOrCreate()


# Carga de datos

En la siguiente celda realizamos la carga de los datos del dataset de Movielens. en este caso vamos a cargar 3 de las tablas:

* ratings
* movies
* tags

Para esto definimos primero los schemas:

In [ ]:
ratingsDf = spark.read.csv("./ml-latest-small/ratings.csv")
moviesDf = spark.read.csv("./ml-latest-small/movies.csv")
tagsDf = spark.read.csv("./ml-latest-small/tags.csv")

Revisamos primero el dataframe de ratings

In [ ]:
ratingsDf.head(2)

Vemos que tiene header, y los tipos de datos, incluyendo el ultimo que es un timestamp. Podemos definir el schema:

In [ ]:
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])

In [ ]:
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("./ml-latest-small/ratings.csv")
ratingsDf.head(2)

In [ ]:
ratingsDf.printSchema()

In [ ]:
movies_schema  = StructType(fields=[
    StructField("movieId",IntegerType(),True), 
    StructField("title",StringType(),True),
    StructField("genres",StringType(),True)
])

moviesDf = spark.read\
    .option("header", True)\
    .schema(movies_schema)\
    .csv("./ml-latest-small/movies.csv")
moviesDf.printSchema()

In [ ]:
moviesDf.show()

Notamos dos cosas:

1. En la columna `genres` vienen multiples valores separados por `|`
1. En la columna `title` viene tanto el nombre de la palicula como el año en que salió

Queremos separar esto.

In [ ]:
from pyspark.sql.functions import split, col

# La función spli separa una columna en un arreglo
moviesDf.select(split(col("genres"),"\|").alias("genresSplit")).show()


O de otra forma:

In [ ]:
moviesDf.withColumn("genresSplit", split(moviesDf["genres"],"\|")).show()


In [ ]:
moviesDfSplit = moviesDf.withColumn("genresSplit", split(moviesDf["genres"],"\|"))
moviesDfSplit.printSchema()

No es necesario mantener ambas columnas, entonces es posible eliminar la columna `genres`

In [ ]:
moviesDfSplit.drop("genres")

Ahora revisamos el dataframe

In [ ]:
moviesDfSplit.printSchema()

Los dataframes son inmutables, por lo que vimos anteriormente fue un nuevo dataframe. Para conservar el cambio tenemos que asignarlo para ver el cambio.

In [ ]:
moviesDfSplit = moviesDfSplit.drop("genres")
moviesDfSplit.printSchema()

Ahora sucede lo mismo con la columna title. Podemos extrar el nombre de la película y el año y crear columnas especificas para cada dato. Usamos expresiones regulares con la funcion [regexp_extract](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.regexp_extract.html#pyspark.sql.functions.regexp_extract)

In [ ]:
from pyspark.sql.functions import regexp_extract, col

moviesDfSplit.withColumn("year", regexp_extract(moviesDf["title"],"^.+\(([0-9]+)\)$",1)).show()



In [ ]:
moviesDfSplit.withColumn("year", regexp_extract(moviesDf["title"],"^.+\(([0-9]+)\)$",1)).printSchema()


In [ ]:
moviesDfSplit.withColumn("year", regexp_extract(moviesDf["title"],"^.+\(([0-9]+)\)$",1).cast(IntegerType())).show()


In [ ]:
moviesDfSplit.withColumn("year", regexp_extract(moviesDf["title"],"^.+\(([0-9]+)\)$",1).cast(IntegerType())).printSchema()


In [ ]:
moviesDfSplit = moviesDfSplit\
                .withColumn(\
                            "year",\
                            regexp_extract(\
                                           moviesDf["title"],\
                                           "^.+\(([0-9]+)\)$",\
                                           1)\
                            .cast(IntegerType()))


Existen muchas funciones similares que se pueden utilizar, la documentación está en [Spark SQL Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)

In [ ]:
moviesDfSplit.show()

Hacemos lo mismo para obtener el titulo

In [ ]:
moviesDfSplit=moviesDfSplit\
                .withColumn(\
                            "title_temp",\
                            regexp_extract(\
                                           _["title"],\
                                           "^(.+?) \([0-9]+\)$",\
                                           1))\
                .drop('title')\
                .withColumnRenamed("title_temp","title")
moviesDfSplit.printSchema()

In [ ]:
moviesDfSplit.show()

Por ultimo hacemos lo mismo con la tabla de tags

In [ ]:
tagsDf.show()

In [ ]:
tags_schema  = StructType(fields=[    
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True), 
    StructField("tag",StringType(),True),
    StructField("timestamp",LongType(),True)
])

tagsDf = spark.read\
    .option("header", True)\
    .schema(tags_schema)\
    .csv("./ml-latest-small/tags.csv")

tagsDf.show()

In [ ]:
tagsDf.printSchema()

In [ ]:
tagsDf.describe().show()

Finalmente, convertimos el timestamp en una fecha:

In [ ]:
from pyspark.sql.functions import from_unixtime


tags_schema  = StructType(fields=[    
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True), 
    StructField("tag",StringType(),True),
    StructField("timestamp",LongType(),True)
])

tagsDf = spark.read\
    .option("header", True)\
    .schema(tags_schema)\
    .csv("./ml-latest-small/tags.csv")


tagsDf=tagsDf\
        .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')

tagsDf.show()